In [ ]:
import xarray as xr
import xesmf as xe

from arguments import ArgumentsSpring2018

In [ ]:
args = ArgumentsSpring2018()

In [ ]:
time_slice = slice(args.start_time, args.end_time)
ds_data = xr.open_mfdataset(list(args.roho160_files)).sel(ocean_time=time_slice)
ds_grid = xr.open_dataset(args.input_grid_file)

### Interpolate (regrid) temperature from 160 m to 800 m

Prepare variables and resample from 160 to 800 meters domain.
Description of xesmf is unclear for me, so renaming is to follow the examples
at `https://xesmf.readthedocs.io/en/latest/notebooks/Curvilinear_grid.html`.

In [ ]:
ds_grid = ds_grid.rename({"lon_rho": "lon", "lat_rho": "lat"})
ds_data = ds_data.rename({"lon_rho": "lon", "lat_rho": "lat"})

In [ ]:
regridder = xe.Regridder(ds_data, ds_grid, "bilinear", unmapped_to_nan=True)
da_sst = regridder(ds_data['temp']).isel(s_rho=-1)

Visual check

In [ ]:
da_sst = da_sst / ds_grid.mask_rho  # type: ignore ; exclude values outside the sea

In [ ]:
ds_data['temp'].isel(ocean_time=-1, s_rho=-1).plot(figsize=(14, 7))  # type: ignore

In [ ]:
da_sst.isel(ocean_time=-1).plot(figsize=(14, 7))  # type: ignore